# Module 9: Locks and Blocking

**Goal**: Understanding the price of safety. If two people want to modify the same row, one of them must wait.

In the previous chapter, we saw "soft" conflicts (reading changing data). Now we deal with "hard" conflicts: Writing. Postgres uses a system where Readers don't block Writers, and Writers don't block Readers (thanks to MVCC).

However, Writers ALWAYS block Writers. If we both try to change Alice's balance, the database forces us into a single-file line.

Let's break the physics of the "Lock".

-----

## 1. Setup
We need a fresh `accounts` table. We will also use Python's `threading` library to simulate two users clicking "Update" at the exact same nanosecond.

In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import time
import threading

# DB Params
DB_PARAMS = {
    "host": "db_int_opt",
    "port": 5432,
    "user": "admin",
    "password": "password",
    "dbname": "db_int_opt"
}

def reset_chapter9():
    with psycopg2.connect(**DB_PARAMS) as conn:
        with conn.cursor() as cur:
            cur.execute("DROP TABLE IF EXISTS accounts;")
            cur.execute("CREATE TABLE accounts (id SERIAL PRIMARY KEY, name TEXT, balance INT);")
            cur.execute("INSERT INTO accounts (name, balance) VALUES ('Alice', 1000), ('Bob', 1000);")
        conn.commit()
    print("module 9 Environment Ready.")

reset_chapter9()

----

## 2. Experiment 9.1: The Standoff (Row-Level Locking)
**The Concept**: When User A updates a row, they acquire a **Row Exclusive Lock**. Until User A commits (or rolls back), that row is "checked out." If User B tries to update the same row, User B is put to sleep (blocked).

**Hypothesis**: We will launch a "Holder" thread that locks Alice for 3 seconds. We will launch a "Waiter" thread 1 second later that tries to update Alice.
- **Question**: Will the "Waiter" finish instantly, or will its execution time be forced to match the "Holder's" delay?

In [ ]:
reset_chapter9()

def hold_lock():
    try:
        with psycopg2.connect(**DB_PARAMS) as conn:
            with conn.cursor() as cur:
                print("[Holder] Acquiring Lock on Alice...")
                cur.execute("UPDATE accounts SET balance = balance + 100 WHERE name = 'Alice';")
                print("[Holder] Lock Acquired. Sleeping for 3 seconds (holding lock)...")
                time.sleep(3)
                conn.commit()
                print("[Holder] Committed and Released Lock.")
    except Exception as e:
        print(f"[Holder] Error: {e}")

def wait_for_lock():
    try:
        # Give holder a head start
        time.sleep(1) 
        
        start_time = time.time()
        print("[Waiter] Trying to update Alice...")
        
        with psycopg2.connect(**DB_PARAMS) as conn:
            with conn.cursor() as cur:
                # This line should HANG until Holder commits
                cur.execute("UPDATE accounts SET balance = balance + 100 WHERE name = 'Alice';")
                conn.commit()
        
        end_time = time.time()
        duration = end_time - start_time
        print(f"[Waiter] Finally updated Alice! Waited {duration:.2f} seconds.")
        return duration
    except Exception as e:
        print(f"[Waiter] Error: {e}")
        return 0

# Run threads
t1 = threading.Thread(target=hold_lock)
t2 = threading.Thread(target=wait_for_lock)

t1.start()
t2.start()
t1.join()
t2.join()

#### Step 3: Visualization (No graph needed here, the console output showing Waited ~2.00 seconds is the proof).

**The Physics**: The "Waiter" didn't do any complex math. It spent 2 seconds simply waiting for a signal from the Lock Manager saying "Row #1 is free." This is the primary cause of slowness in high-concurrency apps (not CPU, not Disk, but Contention).


---

## 3. Experiment 9.2: The Deadlock (The Death Hug)
**The Concept**: What if two users lock two different resources in opposite orders?
- User A: Locks Alice. Wants Bob.
- User B: Locks Bob. Wants Alice.

Neither can proceed. Neither can release their current lock. They will wait until the end of time. Fortunately, the database has a Deadlock Detector that wakes up every ~1 second to check for cycles in the dependency graph. It will kill one process to save the other.

Hypothesis: We will purposely engineer a deadlock. We expect Python to throw a `DeadlockDetected` exception.

In [ ]:
reset_chapter9()

barrier = threading.Barrier(2) # To sync the threads perfectly

def txn_a():
    with psycopg2.connect(**DB_PARAMS) as conn:
        with conn.cursor() as cur:
            try:
                # 1. Lock Alice
                cur.execute("UPDATE accounts SET balance = 100 WHERE name = 'Alice';")
                print("[Txn A] Locked Alice. Waiting...")
                barrier.wait() # Wait for B to lock Bob
                
                # 2. Try to Lock Bob (B has it)
                print("[Txn A] Trying to lock Bob...")
                cur.execute("UPDATE accounts SET balance = 100 WHERE name = 'Bob';")
                conn.commit()
            except Exception as e:
                print(f"\n[Txn A] ☠️ KILLED BY DATABASE: {e}")

def txn_b():
    with psycopg2.connect(**DB_PARAMS) as conn:
        with conn.cursor() as cur:
            try:
                # 1. Lock Bob
                cur.execute("UPDATE accounts SET balance = 200 WHERE name = 'Bob';")
                print("[Txn B] Locked Bob. Waiting...")
                barrier.wait() # Wait for A to lock Alice
                
                # 2. Try to Lock Alice (A has it)
                print("[Txn B] Trying to lock Alice...")
                cur.execute("UPDATE accounts SET balance = 200 WHERE name = 'Alice';")
                conn.commit()
            except Exception as e:
                print(f"\n[Txn B] ☠️ KILLED BY DATABASE: {e}")

t_a = threading.Thread(target=txn_a)
t_b = threading.Thread(target=txn_b)

t_a.start()
t_b.start()
t_a.join()
t_b.join()

**The Physics**: The database engine maintains a "Wait-For Graph" in memory. A -> B and B -> A. When the cycle detector runs, it sees the circle. It arbitrarily picks a victim (usually the most recent one), rolls it back, and sends a 40P01: deadlock_detected error.

---

## 4. Experiment 9.3: MVCC (Why Readers Don't Wait)
**The Concept**: In old databases (like early SQL Server or DB2), if I was writing to a row, you couldn't read it. You would get blocked. Postgres uses MVCC (Multi-Version Concurrency Control). While I am updating "Alice", the old version of Alice still exists on the disk page.

**Hypothesis**: We will lock Alice (update without commit). While the lock is held, we will try to `SELECT * FROM accounts`.
- **Result**: The read should be instant and return the old value.

In [ ]:
reset_chapter9()

def blocking_writer():
    with psycopg2.connect(**DB_PARAMS) as conn:
        with conn.cursor() as cur:
            print("[Writer] Updating Alice to $9999 (No Commit yet)...")
            cur.execute("UPDATE accounts SET balance = 9999 WHERE name = 'Alice';")
            time.sleep(2)
            # We never commit, just close (Rollback).
            print("[Writer] Rolling back changes.")

def nimble_reader():
    time.sleep(0.5) # Wait for writer to grab lock
    start = time.time()
    with psycopg2.connect(**DB_PARAMS) as conn:
        with conn.cursor() as cur:
            print("[Reader] Reading Alice...")
            cur.execute("SELECT balance FROM accounts WHERE name = 'Alice';")
            val = cur.fetchone()[0]
            print(f"[Reader] Read Success! Balance: ${val} (Time: {time.time()-start:.4f}s)")

t_w = threading.Thread(target=blocking_writer)
t_r = threading.Thread(target=nimble_reader)

t_w.start()
t_r.start()
t_w.join()
t_r.join()

#### Step 4: The Physics
 The Reader did not block. It saw `$1000`, even though the Writer had temporarily set it to `$9999` in memory. This is why Postgres is excellent for mixed workloads (Reporting + Transactional) compared to locking-based engines.

----

## Key Takeaways
1. **Writes Block Writes**: Two objects cannot occupy the same space at the same time.
2. **Deadlocks**: If you don't agree on a locking order (always lock A then B), you will kill your own processes.
3. **Readers are Ghosts**: In MVCC, readers float through walls (locks) and see the past.